In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import progressbar

In [4]:
mnist_data = input_data.read_data_sets('./MNIST_data', one_hot=True)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


## build model

In [23]:
batch_size = 60
hidden1_dim = 100
hidden2_dim = 120
input_dim = 784
output_dim = 10
def mlp(input, hidden1_dim, hidden2_dim):
    with tf.name_scope('hidden1'):
        w_values = tf.truncated_normal(shape=[input_dim, hidden1_dim], stddev=1./np.sqrt(float(input_dim)))
        weights = tf.Variable(w_values, name='weights')
        b_values = tf.zeros(shape=[hidden1_dim])
        biases = tf.Variable(b_values, name='biases')
        hidden1 = tf.nn.relu(tf.matmul(input, weights) + biases)
    
    with tf.name_scope('hidden2'):
        w_values = tf.truncated_normal(shape=[hidden1_dim, hidden2_dim], stddev=1./np.sqrt(float(hidden1_dim)))
        weights = tf.Variable(w_values, name='weights')
        b_values = tf.zeros(shape=[hidden2_dim])
        biases = tf.Variable(b_values, name='biases')
        hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)
    
    with tf.name_scope('softmax_linear'):
        w_values = tf.truncated_normal(shape=[hidden2_dim, output_dim], stddev=1./np.sqrt(float(hidden2_dim)))
        weights = tf.Variable(w_values, name='weights')
        b_values = tf.zeros(shape=[output_dim])
        biases = tf.Variable(b_values, name='biases')
        output = tf.matmul(hidden2, weights) + biases
    
    return output

def get_loss(output, labels):
    xentropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output, labels))
    return xentropy

def get_eval(output, labels):
    accu = tf.equal(tf.argmax(output, 1), tf.argmax(labels, 1))
    accu = tf.reduce_mean(tf.cast(accu, tf.float32))
    return accu

## training model

In [24]:
def get_train_op(loss):
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train_op = optimizer.minimize(loss)
    return train_op

def valid_eval(sess, test_data, loss, accu, input, labels, batch_size=32):
    num_batches = test_data.num_examples // batch_size
    loss_list, accu_list = [], []
    for ii in range(num_batches):
        batch_x, batch_y = test_data.next_batch(batch_size)
        feed_dict = {input: batch_x, labels: batch_y}
        loss_value, accu_value = sess.run([loss, accu], feed_dict=feed_dict)
        loss_list.append(loss_value)
        accu_list.append(accu_value)
    return np.mean(loss_list), np.mean(accu_list)

def train_model(sess, input, labels, train_op, loss, accu):
    loss_list, accu_list = [], []
    for ii in range(n_epochs):
        bar = progressbar.ProgressBar()
        for jj in bar(range(steps_per_epoch)):
            batch_x, batch_y = mnist_data.train.next_batch(batch_size)
            feed_dict = {input: batch_x, labels: batch_y}
            sess.run([train_op], feed_dict = feed_dict)
        loss_value, accu_value = valid_eval(sess, mnist_data.validation, loss, accu, input, labels)
        loss_list.append(loss_value)
        accu_list.append(accu_value)
        print ("epoch: {}, loss: {:.3f}, accu: {:.3f}".format(ii, loss_value, accu_value))
    return loss_list, accu_list

In [25]:
n_epochs = 10
steps_per_epoch = mnist_data.train.num_examples // batch_size
input = tf.placeholder(tf.float32, shape=[None, input_dim], name='input')
labels = tf.placeholder(tf.float32, shape=[None, output_dim], name='output')


output = mlp(input, hidden1_dim, hidden2_dim)
loss = get_loss(output, labels)
accu = get_eval(output, labels)
train_op = get_train_op(loss)


sess = tf.Session()
sess.run(tf.global_variables_initializer())

'''
loss_list, accu_list = train_model(sess, input, labels, train_op, loss, accu)

plt.figure()
plt.subplot(211)
plt.scatter(range(len(loss_list)), loss_list, label='loss')
plt.legend()
plt.subplot(212)
plt.scatter(range(len(accu_list)), accu_list, label='accu')
plt.legend()
plt.show()
'''
tf.global_variables()

In [14]:
tf.global_variables()